# Traffic Violation RAG System
In this exam, you will implement a Retrieval-Augmented Generation (RAG) system that uses a language model and a vector database to answer questions about traffic violations. The goal is to generate answers with relevant data based on a dataset of traffic violations and fines.

Here are helpful resources:
* [LangChain](https://www.langchain.com/)
* [groq cloud documentation](https://console.groq.com/docs/models)
* [LangChain HuggingFace](https://python.langchain.com/docs/integrations/text_embedding/sentence_transformers/)
* [Chroma Vector Store](https://python.langchain.com/docs/integrations/vectorstores/chroma/)
* [Chroma Website](https://docs.trychroma.com/getting-started)
* [ChatGroq LangChain](https://python.langchain.com/docs/integrations/chat/groq/)
* [LLM Chain](https://api.python.langchain.com/en/latest/chains/langchain.chains.llm.LLMChain.html#langchain.chains.llm.LLMChain)

Dataset [source](https://www.moi.gov.sa/wps/portal/Home/sectors/publicsecurity/traffic/contents/!ut/p/z0/04_Sj9CPykssy0xPLMnMz0vMAfIjo8ziDTxNTDwMTYy83V0CTQ0cA71d_T1djI0MXA30gxOL9L30o_ArApqSmVVYGOWoH5Wcn1eSWlGiH1FSlJiWlpmsagBlKCQWqRrkJmbmqRqUZebngB2gUJAKdERJZmqxfkG2ezgAhzhSyw!!/)

Some installs if needed:
```python
!pip install langchain_huggingface langchain langchain-community langchain_chroma Chroma langchain_groq LLMChain
```

In [ ]:
# !kaggle datasets download -d khaledzsa/dataset
# !unzip dataset.zip

Dataset URL: https://www.kaggle.com/datasets/khaledzsa/dataset
License(s): unknown
dataset.zip: Skipping, found more recently modified local copy (use --force to force download)
Archive:  dataset.zip
replace Dataset.csv? [y]es, [n]o, [A]ll, [N]one, [r]ename: 

## Step 1: Install Required Libraries

To begin, install the necessary libraries for this project. The libraries include `LangChain` for building language model chains, and `Chroma` for managing a vector database.

In [ ]:
!pip install langchain-huggingface langchain langchain-community langchain-chroma Chroma langchain-groq

## Step 2: Load the Traffic Violations Dataset

You are provided with a dataset of traffic violations. Load the CSV file into a pandas DataFrame and preview the first few rows of the dataset using `.head()`. You can also try and see the dataset's characteristics.

In [70]:
pip install pyarabic

In [71]:
import pandas as pd
df = pd.read_csv('/content/Dataset.csv')
df.head()

,المخالفة,الغرامة
0,قيادة المركبة في الأسواق التي لا يسمح بالقيادة...,الغرامة المالية 100 - 150 ريال
1,ترك المركبة مفتوحة وفي وضع التشغيل بعد مغادرتها.,الغرامة المالية 100 - 150 ريال
2,عدم وجود تأمين ساري للمركبة.,الغرامة المالية 100 - 150 ريال
3,عبور المشاة للطرق من غير الأماكن المخصصة لهم.,الغرامة المالية 100 - 150 ريال
4,عدم تقيد المشاة بالإشارات الخاصة بهم.,الغرامة المالية 100 - 150 ريال


In [73]:
import pyarabic.araby as araby
from pyarabic.araby import strip_harakat
def preprocesing(df):
  df['المخالفة']=df['المخالفة'].apply(lambda x: araby.strip_harakat(x))
  df['المخالفة']=df['المخالفة'].apply(lambda x: araby.strip_tashkeel(x))
  df['المخالفة']=df['المخالفة'].apply(lambda x: araby.strip_tatweel(x))
  df['الغرامة']=df['الغرامة'].apply(lambda x: araby.strip_harakat(x))
  df['الغرامة']=df['الغرامة'].apply(lambda x: araby.strip_tashkeel(x))
  df['الغرامة']=df['الغرامة'].apply(lambda x: araby.strip_tatweel(x))
  return df
  print(df)


In [74]:
df.isnull().sum()

,0
المخالفة,0
الغرامة,0


## Step 3: Create Markdown Content from the Dataset

For each traffic violation in the dataset, you will generate markdown text that describes the violation and the associated fine. Create a loop to iterate through the dataset and store the generated markdown in a list. Each fine should look like this:

**المخالفة** - الغرامة

In [75]:
df.duplicated().sum()

0

In [76]:
import os
directory='data/markdown_files'
os.makedirs(directory,exist_ok=True)


In [ ]:
markdown_files=[]
for i in range(0,len(df)):
  title=df['المخالفة'].iloc[i]
  content=df['الغرامة'].iloc[i]
  markdown_content=f"**{title}**{content}"
  markdown_files.append(markdown_content)
  with open(f'{directory}/{i}.md','w') as file:
    file.write(markdown_content)
print(df.columns)

Index(['المخالفة', 'الغرامة'], dtype='object')


In [ ]:
markdown_content

'**المجازفة بعبور الأودية والشعاب أثناء جريانها.**الغرامة المالية 5000 - 10000 ريال'

In [ ]:
markdown_files

['**قيادة المركبة في الأسواق التي لا يسمح بالقيادة فيها.**الغرامة المالية 100 - 150 ريال',
 '**ترك المركبة مفتوحة وفي وضع التشغيل بعد مغادرتها.**الغرامة المالية 100 - 150 ريال',
 '**عدم وجود تأمين ساري للمركبة.**الغرامة المالية 100 - 150 ريال',
 '**عبور المشاة للطرق من غير الأماكن المخصصة لهم.**الغرامة المالية 100 - 150 ريال',
 '**عدم تقيد المشاة بالإشارات الخاصة بهم.**الغرامة المالية 100 - 150 ريال',
 '**وقوف المركبة في أماكن غير مخصصة للوقوف.**الغرامة المالية 100 - 150 ريال',
 '**عدم إعطاء أفضلية المرور للمشاة أثناء عبورهم في المسارات المخصصة لهم.**الغرامة المالية 100 - 150 ريال',
 '**عدم استخدام إشارة الالتفاف عند التحول لليمين أو اليسار أو التجاوز أو تغيير المسار.**الغرامة المالية 150 - 300 ريال',
 '**الرجوع إلى الخلف في الطريق العام لمسافة تزيد على عشرين متراً.**الغرامة المالية 150 - 300 ريال',
 '**قيام سائق الدراجة الآلية أو العادية - أو ما في حكمهما - بالتعلق بأي مركبة أخرى، أو سحب أو حمل أشياء تعرض مستخدمي الطريق للخطر.**الغرامة المالية 150 - 300 ريال',
 '**مخالفة قواعد استعمال

In [ ]:
with open(os.path.join(directory,'0.md'),'r', encoding='utf_8') as file:
  markdown_content=file.read()
  print(markdown_content)


**قيادة المركبة في الأسواق التي لا يسمح بالقيادة فيها.**الغرامة المالية 100 - 150 ريال


## Step 4: Chunk the Markdown Data

Using LangChain's `RecursiveCharacterTextSplitter`, split the markdown texts into smaller chunks that will be stored in the vector database.

In [ ]:
from langchain.text_splitter import RecursiveCharacterTextSplitter
text_splitter=RecursiveCharacterTextSplitter(chunk_size=500,chunk_overlap=100)
documents=text_splitter.create_documents(markdown_files)

In [ ]:
print(f'Now you have {len(documents)} documents')

Now you have 104 documents


In [ ]:
print(documents[7].page_content)

**عدم استخدام إشارة الالتفاف عند التحول لليمين أو اليسار أو التجاوز أو تغيير المسار.**الغرامة المالية 150 - 300 ريال


In [ ]:
print(documents)

[Document(metadata={}, page_content='**قيادة المركبة في الأسواق التي لا يسمح بالقيادة فيها.**الغرامة المالية 100 - 150 ريال'), Document(metadata={}, page_content='**ترك المركبة مفتوحة وفي وضع التشغيل بعد مغادرتها.**الغرامة المالية 100 - 150 ريال'), Document(metadata={}, page_content='**عدم وجود تأمين ساري للمركبة.**الغرامة المالية 100 - 150 ريال'), Document(metadata={}, page_content='**عبور المشاة للطرق من غير الأماكن المخصصة لهم.**الغرامة المالية 100 - 150 ريال'), Document(metadata={}, page_content='**عدم تقيد المشاة بالإشارات الخاصة بهم.**الغرامة المالية 100 - 150 ريال'), Document(metadata={}, page_content='**وقوف المركبة في أماكن غير مخصصة للوقوف.**الغرامة المالية 100 - 150 ريال'), Document(metadata={}, page_content='**عدم إعطاء أفضلية المرور للمشاة أثناء عبورهم في المسارات المخصصة لهم.**الغرامة المالية 100 - 150 ريال'), Document(metadata={}, page_content='**عدم استخدام إشارة الالتفاف عند التحول لليمين أو اليسار أو التجاوز أو تغيير المسار.**الغرامة المالية 150 - 300 ريال'), Document

## Step 5: Generate Embeddings for the Documents

Generate embeddings for the chunks of text using HuggingFace's pre-trained Arabic language model. These embeddings will be stored in a `Chroma` vector store.

In [ ]:
from langchain_community.embeddings.sentence_transformer import SentenceTransformerEmbeddings
from langchain.vectorstores import Chroma
embeddings_function=SentenceTransformerEmbeddings(model_name='Omartificial-Intelligence-Space/GATE-AraBert-v0')
db=Chroma.from_documents(documents,embeddings_function,persist_directory='db')


In [79]:
result=db.similarity_search_with_relevance_scores("مخالفة عدم ربط حزام الامان", k=4)


<ipython-input-79-20bdfaa39633>:1: UserWarning: Relevance scores must be between 0 and 1, got [(Document(metadata={}, page_content='**عدم ربط حزام الأمان.**الغرامة المالية 150 - 300 ريال'), -247.51777956768595), (Document(metadata={}, page_content='**عدم ربط حزام الأمان.**الغرامة المالية 150 - 300 ريال'), -247.51777956768595), (Document(metadata={}, page_content='**عدم ربط حزام الأمان.**الغرامة المالية 150 - 300 ريال'), -247.51777956768595), (Document(metadata={}, page_content='**عدم اتخاذ الاحتياطات اللازمة عند إيقاف المركبة في حالات الطوارئ على الطرق العامة.**الغرامة المالية 300 - 500 ريال'), -271.31522005430094)]
  result=db.similarity_search_with_relevance_scores("مخالفة عدم ربط حزام الامان", k=4)


## Step 6: Define the RAG Prompt Template

Define a custom prompt template in Arabic to retrieve traffic violation-related answers based on the context. Ensure the template greets the user first, states that the information provided could be incorrect, and advises the user to visit the traffic initiative website to verify. Additionally, provide the user with advice in Arabic, ensuring it stays within the given context.

In [ ]:
from langchain.prompts import PromptTemplate
PROMOT_TEMPLATE="""
أجب بناءً على السياق فقط وليس بناءً على معرفتك، واجعل إجابتك قصيرة وواضحة.
---
السياق::{context}
---
السؤال::{question}
"""

prompt=PromptTemplate(template=PROMOT_TEMPLATE,input_variables=['context','question'])


## Step 7: Initialize the Language Model

Initialize the language model using the Groq API. Set up the model with a specific configuration, including the API key, temperature setting, and model name.

In [ ]:
from langchain_groq import ChatGroq

groq_api_key='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'
llm=ChatGroq(temperature=0,groq_api_key=groq_api_key,model_name="llama-3.1-70b-versatile")


Now, you will create an LLM Chain that combines the language model and the prompt template you defined. This chain will be used to generate responses based on the retrieved context.

## Step 8: Create the LLM Chain

In [80]:
from langchain.chains import LLMChain
MODEL=LLMChain(llm=llm,prompt=prompt,
               verbose=True)

## Step 9: Implement the Query Function

Create a function `query_rag` that will take a user query as input, retrieve relevant context from the vector store, and use the language model to generate a response based on that context.

In [ ]:
from decimal import Context
def query_rag(query: str):
  similarity_search_results=db.similarity_search_with_score(query, k=1)
  context_text=""

  for result in similarity_search_results:
   if result[1]>0.7:
      context_text=context_text+result[0].page_content +'\n\n'
  rag_response=MODEL.invoke({"context":context_text,"question":query})
  return rag_response


## Step 10: Inference - Running Queries in the RAG System

In this final step, you will implement an inference pipeline to handle real-time queries. You will allow the system to retrieve the most relevant violations and fines based on a user's input and generate a response.

1. Inference Workflow:

  * The user inputs a query (e.g., "ماهي عقوبة عدم الوقوف وقوفاً تاماً عند إشارة؟").
  * The system searches for the most relevant context from the traffic violation vector store.
  * It generates an answer and advice based on the context.

2. Goal:
  * Run the inference to answer questions based on the traffic violation dataset.

In [82]:
result=query_rag("ماهي عقوبة عدم الوقوف وقوفاً تاماً عند إشارة؟")
print(result)

Prompt after formatting:

أجب بناءً على السياق فقط وليس بناءً على معرفتك، واجعل إجابتك قصيرة وواضحة.
---
السياق::**عدم الوقوف تماماً عند إشارة قف.**الغرامة المالية 500 - 900 ريال


---
السؤال::ماهي عقوبة عدم الوقوف وقوفاً تاماً عند إشارة؟


> Finished chain.
{'context': '**عدم الوقوف تماماً عند إشارة قف.**الغرامة المالية 500 - 900 ريال\n\n', 'question': 'ماهي عقوبة عدم الوقوف وقوفاً تاماً عند إشارة؟', 'text': 'الغرامة المالية 500 - 900 ريال.'}


In [83]:
result=query_rag("ماهي غرامة عدم ربط حزام الامان؟")
print(result)

Prompt after formatting:

أجب بناءً على السياق فقط وليس بناءً على معرفتك، واجعل إجابتك قصيرة وواضحة.
---
السياق::**عدم ربط حزام الأمان.**الغرامة المالية 150 - 300 ريال


---
السؤال::ماهي غرامة عدم ربط حزام الامان؟


> Finished chain.
{'context': '**عدم ربط حزام الأمان.**الغرامة المالية 150 - 300 ريال\n\n', 'question': 'ماهي غرامة عدم ربط حزام الامان؟', 'text': '150 - 300 ريال'}
